In [1]:
import tensorflow as tf
import numpy as np
import setGPU

setGPU: Setting GPU to: 0


In [2]:
tf.__version__

'2.1.0'

In [3]:
t = tf.range(0., 10.)
print(t)
t = t * 2
print(t)
t = t + 1
print(t)

tf.Tensor([0. 1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(10,), dtype=float32)
tf.Tensor([ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.], shape=(10,), dtype=float32)
tf.Tensor([ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.], shape=(10,), dtype=float32)


In [4]:
@tf.function
def my_func(t):
    print("new graph created", t.dtype, t.shape)
    t = t * 2
    print(t)
    t = t + 1
    print(t)
    return t

t = tf.range(0., 10.)
my_func(t)

new graph created <dtype: 'float32'> (10,)
Tensor("mul:0", shape=(10,), dtype=float32)
Tensor("add:0", shape=(10,), dtype=float32)


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.], dtype=float32)>

In [5]:
print(t.numpy())

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [6]:
my_func(tf.range(10., 20.))

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([21., 23., 25., 27., 29., 31., 33., 35., 37., 39.], dtype=float32)>

In [7]:
my_func(tf.range(10., 21.))
print("---")
my_func(tf.range(10, 20, dtype=tf.int32))

new graph created <dtype: 'float32'> (11,)
Tensor("mul:0", shape=(11,), dtype=float32)
Tensor("add:0", shape=(11,), dtype=float32)
---
new graph created <dtype: 'int32'> (10,)
Tensor("mul:0", shape=(10,), dtype=int32)
Tensor("add:0", shape=(10,), dtype=int32)


<tf.Tensor: shape=(10,), dtype=int32, numpy=array([21, 23, 25, 27, 29, 31, 33, 35, 37, 39], dtype=int32)>

In [3]:
import dadrah.playground.test_gradient_tape as tegrta

In [4]:
from importlib import reload
reload(tegrta)
model = tegrta.make_model()
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
feature_normalization (Featu (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 20)                40        
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_______________________________________________________________

In [5]:
targets = np.arange(0.,10.)
predictions = np.random.random(size=10)*10
print(targets)
print(predictions)
print(targets-predictions)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
[5.74078807 4.49240333 4.83217408 6.26803847 8.1283458  5.85762376
 4.37126549 7.37263836 9.57592416 6.51941307]
[-5.74078807 -3.49240333 -2.83217408 -3.26803847 -4.1283458  -0.85762376
  1.62873451 -0.37263836 -1.57592416  2.48058693]


In [6]:
quantile_loss = tegrta.make_quantile_loss(np.mean(targets), np.var(targets))

In [7]:
print(quantile_loss(targets, predictions))

tf.Tensor(
[0.62862426 0.49166458 0.51352044 0.64498567 0.81889519 0.57970177
 0.41894473 0.70696081 0.91516817 0.59739585], shape=(10,), dtype=float64)


In [8]:
print(quantile_loss(targets, predictions))

tf.Tensor(
[0.62862426 0.49166458 0.51352044 0.64498567 0.81889519 0.57970177
 0.41894473 0.70696081 0.91516817 0.59739585], shape=(10,), dtype=float64)


In [9]:
optimizer = tf.keras.optimizers.Adam(0.005)

In [10]:
x = tf.random.uniform([300, 1], minval=1, maxval=3000, dtype=tf.float32)

In [11]:
x.shape

TensorShape([300, 1])

In [12]:
y = tf.random.uniform([300,1], minval=1, maxval=10, dtype=tf.float32)

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(32)

In [14]:
quantile_loss = tegrta.make_quantile_loss(np.mean(y), np.var(y))

In [15]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            predictions = model(x_batch_train, training=True)
            loss_value = quantile_loss(y_batch_train, predictions)
        
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 2 == 0:
            print("Training loss (for one batch) at step {}: {}".format(step, np.sum(loss_value)))
            print("Seen so far: %s samples" % ((step + 1) * 32))



Start of epoch 0
Training loss (for one batch) at step 0: 408.56719970703125
Seen so far: 32 samples
Training loss (for one batch) at step 2: 610.732177734375
Seen so far: 96 samples
Training loss (for one batch) at step 4: 75.10333251953125
Seen so far: 160 samples
Training loss (for one batch) at step 6: 182.886474609375
Seen so far: 224 samples
Training loss (for one batch) at step 8: 160.21493530273438
Seen so far: 288 samples

Start of epoch 1
Training loss (for one batch) at step 0: 151.64012145996094
Seen so far: 32 samples
Training loss (for one batch) at step 2: 97.97803497314453
Seen so far: 96 samples
Training loss (for one batch) at step 4: 38.00904083251953
Seen so far: 160 samples
Training loss (for one batch) at step 6: 340.62115478515625
Seen so far: 224 samples
Training loss (for one batch) at step 8: 57.522300720214844
Seen so far: 288 samples


In [16]:
x_test = tf.random.uniform([300, 1], minval=1, maxval=3000, dtype=tf.float32)

In [17]:
y = model.predict(x_test)

In [18]:
y

array([[1.33943272e+01],
       [8.49972153e+01],
       [6.75440674e+01],
       [3.78126717e+01],
       [2.49080563e+01],
       [7.64242249e+01],
       [4.26838455e+01],
       [7.01208038e+01],
       [3.71308250e+01],
       [9.70166245e+01],
       [7.87173004e+01],
       [3.29775505e+01],
       [8.75404816e+01],
       [2.85773354e+01],
       [8.56708050e+00],
       [4.68315201e+01],
       [6.53647308e+01],
       [4.13506126e+01],
       [7.85710907e+01],
       [8.16198730e+01],
       [2.47511158e+01],
       [7.53266602e+01],
       [3.90911598e+01],
       [7.76971207e+01],
       [6.66692810e+01],
       [8.76261978e+01],
       [3.10476437e+01],
       [2.12336254e+01],
       [1.63540955e+01],
       [9.72009659e+01],
       [2.27431889e+01],
       [1.41934366e+01],
       [4.06639595e+01],
       [7.95788345e+01],
       [2.97757893e+01],
       [6.09283295e+01],
       [2.14148922e+01],
       [9.14779739e+01],
       [7.13663769e+00],
       [2.99947109e+01],


In [19]:
weights = model.get_weights()

In [20]:
model.save('./my_model.h5')

In [21]:
reload(tegrta)
loaded_model = tf.keras.models.load_model('./my_model.h5', custom_objects={'FeatureNormalization': tegrta.FeatureNormalization})

TypeError: __init__() missing 2 required positional arguments: 'mean_input' and 'var_input'

In [27]:
loaded_weights = loaded_model.get_weights()

In [28]:
np.asarray(loaded_weights) == np.asarray(weights)

/afs/cern.ch/user/k/kiwoznia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [29]:
weights[0]

array([[-0.47903162,  0.28525615, -0.06959158, -0.12437147, -0.08485508,
        -0.01661247,  0.4193647 , -0.3637167 ,  0.05655042,  0.22925146,
        -0.45503235,  0.14597094, -0.09571069, -0.19529825, -0.38948673,
        -0.47466895,  0.46109393,  0.4271383 ,  0.15025072, -0.11483219]],
      dtype=float32)

In [30]:
loaded_weights[0]

array([[-0.47903162,  0.28525615, -0.06959158, -0.12437147, -0.08485508,
        -0.01661247,  0.4193647 , -0.3637167 ,  0.05655042,  0.22925146,
        -0.45503235,  0.14597094, -0.09571069, -0.19529825, -0.38948673,
        -0.47466895,  0.46109393,  0.4271383 ,  0.15025072, -0.11483219]],
      dtype=float32)

In [31]:
weights[0] == loaded_weights[0]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]])

In [32]:
for i in range(len(weights)):
    print((np.asarray(weights[i]) == np.asarray(loaded_weights[i])).all())

True
True
True
True
True
True
True
True


In [33]:
(np.asarray(weights) == np.asarray(loaded_weights))

/afs/cern.ch/user/k/kiwoznia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [34]:
np.asarray(weights)

array([array([[-0.47903162,  0.28525615, -0.06959158, -0.12437147, -0.08485508,
        -0.01661247,  0.4193647 , -0.3637167 ,  0.05655042,  0.22925146,
        -0.45503235,  0.14597094, -0.09571069, -0.19529825, -0.38948673,
        -0.47466895,  0.46109393,  0.4271383 ,  0.15025072, -0.11483219]],
      dtype=float32),
       array([ 0.        ,  0.00455843,  0.        ,  0.        ,  0.        ,
        0.        , -0.02051413,  0.        , -0.04273701, -0.00438306,
        0.        , -0.01329798,  0.        ,  0.        ,  0.        ,
        0.        , -0.02088908, -0.00239613,  0.01232171,  0.        ],
      dtype=float32),
       array([[-0.0737097 ,  0.01661929,  0.05339158, -0.3248325 , -0.26583722,
        -0.02814654, -0.16245396,  0.06878287,  0.08197182, -0.19200988,
        -0.04568201,  0.03870448, -0.15482612,  0.17255163,  0.33140427,
        -0.16084497, -0.0499585 , -0.11367702,  0.22446299, -0.12444285],
       [ 0.26927343, -0.15289183, -0.02635756, -0.03964471,

In [35]:
y_loaded = loaded_model.predict(x_test)

In [36]:
(y == y_loaded).all()

True